# Part I. ETL Pipeline for Pre-Processing the Files

This notebook has the objective of covering the whole ETL Pipeline from preprossessing files into the Cassandra Database, and executing some basic queries. For that the code will generate one table structure per each query needed.


## Import libraries
-----


In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

## Environment checks and configurations
------

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/Users/guilherme/Documents/OneDrive/DataAnalytics/Python/ETL Cassandra


## Creation of ETL function
------
Function that iterate through the files list and for each file open the file and append the data into a list. This list is eventually written into the _event_datafile_new.csv_ to be consumed at once into the Cassandra DB

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


## Health check on the extraction function above, to check if the file was loaded properly
------
Expect 6821 rows

In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


## Start connection with Cassandra instance
------

In [5]:
# Make a connection Cassandra instance
from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

# Establish connection
session = cluster.connect()

## Create Keyspace
------

In [6]:
# Keyspace creation
session.execute("""
CREATE KEYSPACE IF NOT EXISTS udacity 
WITH REPLICATION = 
{ 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

## Set session in the created Keyspace
------

In [7]:
# Set Keyspace
session.set_keyspace('udacity')

## Create 3 table structures, one for each query:
-------
* Table _song_info_by_session_ so it is possible to query artists, song title and song's lentgth given a certain sessionId and Item In Session
* Table _artist_sing_by_user_session_ so it is possible to query artist name and song sorted by Item in Session and user first and last name given certain user and session sessionId
* Table _user_name_listened_All_Hands_Against_His_Own_ so it is possible to query First and Last name of users who listened to a specific song
   

In [8]:
# Create one table structure per query need

## Create table structure for query need 1: Retrieve artist, song title and song's length in the music app \ 
## history that was heard during sessionId = 338, and itemInSession = 4
query1_create = """
CREATE TABLE IF NOT EXISTS song_info_by_session (
    sessionid int,
    item_in_session int,
    artist text,
    length float,
    song_title text,
    PRIMARY KEY (
        sessionId, 
        item_in_session
        ))"""
session.execute(query1_create)

## Create table structure for query 2: Retrive only the following: name of artist, song (sorted by itemInSession) \ 
## and user (first and last name) for userid = 10, sessionid = 182
query2_create = """
CREATE TABLE IF NOT EXISTS artist_song_by_user_session (
    userid int,
    sessionid int,
    item_in_session int,
    artist text,
    firstname text,
    lastname text,
    song_title text,
    PRIMARY KEY (
        (userid, 
        sessionid),
        item_in_session
        ))"""
session.execute(query2_create)

## Create table structure for query 3: Retrieve every user name (first and last) in my music app history who listened to\ 
##  the song 'All Hands Against His Own'
query3_create = """
CREATE TABLE IF NOT EXISTS user_name_listened_All_Hands_Against_His_Own (
    song_title text,
    userid int,
    firstname text,
    lastname text,
    PRIMARY KEY (
        song_title, 
        userid    
       ))"""
session.execute(query3_create)

## Ingest data into the data structures
-------
* _query1_ insert the values from the _event_datafile_new.csv_ file into the _song_info_by_session_ table respecting the order of the partition key 

* _query2_ insert the values from the _event_datafile_new.csv_ file into the _artist_song_by_user_session_ table respecting the order of the partition key 

* _query3_ insert the values from the _event_datafile_new.csv_ file into the _user_name_listened_All_Hands_Against_His_Own_ table respecting the order of the partition key 


In [9]:
# Define file name to receive the aggregated data
file = 'event_datafile_new.csv'


# Import individual files information into the aggregated file data 

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        # Define queries 1, 2 and 3 to insert data in the respective structures created above
        query1 = "INSERT INTO song_info_by_session (sessionid, item_in_session, artist, length, song_title)"
        query1 = query1 + "VALUES (%s, %s, %s, %s, %s)"
        query2 = "INSERT INTO artist_song_by_user_session (userid, sessionid, item_in_session, artist, firstname, lastname, song_title)"
        query2 = query2 + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        query3 = "INSERT INTO user_name_listened_All_Hands_Against_His_Own (song_title, userid,  firstname, lastname)"
        query3 = query3 + "VALUES (%s, %s, %s, %s)"
        # Assign each column element for each column in the INSERT statement.
        session.execute(query1, (int(line[8]), int(line[3]),line[0], float(line[5]), line[9]))
        session.execute(query2, (int(line[10]),int(line[8]), int(line[3]), line[0], line[1], line[4], line[9]))
        session.execute(query3, (line[9], int(line[10]),  line[1], line[4]))
        

## Data quality check
-------
Check if the data was loaded correctly in each data structure. Each table should contain thousands of records

In [10]:
# Check if data was included in table songs1
print("Total record of song_info_by_session:")
query = "SELECT COUNT(sessionId) FROM song_info_by_session"
rows = session.execute(query)
for row in rows:
    print(row[0])

# Check if data was included in table songs2
print("Total record of artist_song_by_user_session:")
query = "SELECT COUNT(userid) FROM artist_song_by_user_session"
rows = session.execute(query)
for row in rows:
    print(row[0])

# Check if data was included in table songs3
print("Total record of user_name_listened_All_Hands_Against_His_Own:")
query = "SELECT COUNT(userid) FROM user_name_listened_All_Hands_Against_His_Own"
rows = session.execute(query)
for row in rows:
    print(row[0])

Total record of song_info_by_session:
6820
Total record of artist_song_by_user_session:
6820
Total record of user_name_listened_All_Hands_Against_His_Own:
6618


## Qurery 1
-------
Retrieve the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4.

In order to achieve the result expected for this query, the table structure for this query is composed of a compound key (sessionID and Item In Session)

In [11]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

query1 = "SELECT artist, song_title, length FROM song_info_by_session WHERE sessionId = 338 AND item_in_session = 4"
rows1 = session.execute(query1)
for row1 in rows1:
    print (row1.artist, row1.song_title, row1.length)             

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


## Qurery 2
-------
Retrieve name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In order to achieve the result expected for this query, the table structure for this query is composed of a compound key (sessionID and userid)

In [12]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
query2 = "SELECT artist, song_title, firstname, lastname FROM artist_song_by_user_session WHERE userid = 10 AND sessionid = 182 ORDER BY item_in_session"
rows2 = session.execute(query2)
for row2 in rows2:
    print (row2.artist, row2.song_title, row2.firstname, row2.lastname)    

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


## Qurery 3
-------
Retrieve user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In order to achieve the result expected for this query, the table structure for this query is composed of a partition key and two clustering keys

In [13]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
query3 = "SELECT firstname, lastname FROM user_name_listened_All_Hands_Against_His_Own WHERE song_title = 'All Hands Against His Own' "
rows3 = session.execute(query3)
for row3 in rows3:
    print (row3.firstname, row3.lastname)

Jacqueline Lynch
Tegan Levine
Sara Johnson


## Drop the tables before closing out the sessions
------

In [14]:
## TO-DO: Drop the table before closing out the sessions
rows3 = session.execute('DROP TABLE song_info_by_session')
rows3 = session.execute('DROP TABLE artist_song_by_user_session')
rows3 = session.execute('DROP TABLE user_name_listened_All_Hands_Against_His_Own')

## Shutdown session
-------
End of script

In [15]:
session.shutdown()
cluster.shutdown()